In [1]:
%load_ext nb_black

import os
import sys

sys.path.append(
    os.path.abspath("/ocean/projects/asc170022p/shg121/PhD/ICLR-2022/codebase")
)
import torch.backends.cudnn as cudnn
import random
import time
from collections import OrderedDict
import pickle
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.nn.functional import one_hot
import sklearn.metrics as metrics
import utils
from Explainer.loss_F import loss_fn_kd, entropy_loss
from Explainer.models.Gated_Logic_Net import Gated_Logic_Net
from Explainer.models.explainer import Explainer
from Explainer.models.pi import Pi
from dataset.dataset_cubs import Dataset_cub_for_explainer
from dataset.utils_dataset import get_dataset_with_image_and_attributes
from Explainer.loss_F import loss_fn_kd, entropy_loss, Selective_Distillation_Loss
from Explainer.models.concepts import Conceptizator

import matplotlib.pyplot as plt

<IPython.core.display.Javascript object>

In [2]:
base_lr = 0.01
base_cov = 0.2
pickle_in = open(
    os.path.join(
        f"/ocean/projects/asc170022p/shg121/PhD/ICLR-2022/out/HAM10k/explainer/lr_{base_lr}_epochs_500_temperature-lens_0.7_input-size-pi_2048_cov_{base_cov}_alpha_0.5_selection-threshold_0.5_lambda-lens_0.0001_alpha-KD_0.9_temperature-KD_10.0_hidden-layers_1",
        "iter1",
        "explainer/accuracy",
        "test_explainer_configs.pkl",
    ),
    "rb",
)
args = pickle.load(pickle_in)
n_classes = len(args.labels)
x_to_bool = 0.5
top_k_explanations = 50
concept_names = args.concept_names
print("########################")
print(args.lr[0])
print(args.cov[0])

########################
0.01
0.2


<IPython.core.display.Javascript object>

In [3]:
use_concepts_as_pi_input = True
explainer_init = "none"

experiment_folder = (
    f"lr_{args.lr[0]}_epochs_{args.epochs}_temperature-lens_{args.temperature_lens}"
    f"_input-size-pi_{args.input_size_pi}"
    f"_cov_{args.cov[0]}_alpha_{args.alpha}_selection-threshold_{args.selection_threshold}"
    f"_lambda-lens_{args.lambda_lens}_alpha-KD_{args.alpha_KD}"
    f"_temperature-KD_{float(args.temperature_KD)}_hidden-layers_{len(args.hidden_nodes)}"
)

root = "/ocean/projects/asc170022p/shg121/PhD/ICLR-2022/out/HAM10k"
experiment = f"explainer/{experiment_folder}"
iteration = "iter1"
expert_type = "explainer/accuracy"
output = "g_outputs"


# val
train_tensor_image_1 = torch.load(
    os.path.join(
        root, experiment, iteration, expert_type, output, "train_tensor_images.pt"
    )
)

train_tensor_y_1 = torch.load(
    os.path.join(
        root, experiment, iteration, expert_type, output, "train_tensor_y.pt"
    )
)

train_concept_mask_1 = torch.load(
    os.path.join(root, experiment, iteration, expert_type, output, "train_mask_alpha.pt")
)

test_tensor_image_1 = torch.load(
    os.path.join(
        root, experiment, iteration, expert_type, output, "val_tensor_images.pt"
    )
)

test_tensor_y_1 = torch.load(
    os.path.join(
        root, experiment, iteration, expert_type, output, "val_tensor_y.pt"
    )
)

test_concept_mask_1 = torch.load(
    os.path.join(root, experiment, iteration, expert_type, output, "val_mask_alpha.pt")
)


print("\n\n << Train sizes >>")
print(train_tensor_image_1.size())
print(train_tensor_y_1.size())
print(train_concept_mask_1.size())


print("\n\n << Test sizes >>")
print(test_tensor_image_1.size())
print(test_tensor_y_1.size())
print(test_concept_mask_1.size())




 << Train sizes >>
torch.Size([3108, 3, 299, 299])
torch.Size([3108])
torch.Size([3108, 8])


 << Test sizes >>
torch.Size([799, 3, 299, 299])
torch.Size([799])
torch.Size([799, 8])


<IPython.core.display.Javascript object>

In [8]:
base_lr = 0.01
base_cov = 0.2
pickle_in = open(
    os.path.join(
        f"/ocean/projects/asc170022p/shg121/PhD/ICLR-2022/out/HAM10k/explainer/lr_{base_lr}_epochs_500_temperature-lens_0.7_input-size-pi_2048_cov_{base_cov}_alpha_0.5_selection-threshold_0.5_lambda-lens_0.0001_alpha-KD_0.9_temperature-KD_10.0_hidden-layers_1",
        "cov_0.2",
        "iter2",
        "explainer/accuracy",
        "test_explainer_configs.pkl",
    ),
    "rb",
)
args = pickle.load(pickle_in)
n_classes = len(args.labels)
x_to_bool = 0.5
top_k_explanations = 50
concept_names = args.concept_names
print("########################")
print(args.lr[-1])
print(args.cov[-1])

use_concepts_as_pi_input = True
explainer_init = "none"

experiment_folder = (
    f"lr_{args.lr[0]}_epochs_{args.epochs}_temperature-lens_{args.temperature_lens}"
    f"_input-size-pi_{args.input_size_pi}"
    f"_cov_{args.cov[0]}_alpha_{args.alpha}_selection-threshold_{args.selection_threshold}"
    f"_lambda-lens_{args.lambda_lens}_alpha-KD_{args.alpha_KD}"
    f"_temperature-KD_{float(args.temperature_KD)}_hidden-layers_{len(args.hidden_nodes)}"
)

root = "/ocean/projects/asc170022p/shg121/PhD/ICLR-2022/out/HAM10k"
experiment = f"explainer/{experiment_folder}"
iteration = "iter2"
expert_type = "explainer/accuracy"
output = "g_outputs"

# val
train_tensor_image_2 = torch.load(
    os.path.join(
        root,
        experiment,
        f"cov_{args.cov[-1]}",
        iteration,
        expert_type,
        output,
        "train_tensor_images.pt",
    )
)

train_tensor_y_2 = torch.load(
    os.path.join(
        root,
        experiment,
        f"cov_{args.cov[-1]}",
        iteration,
        expert_type,
        output,
        "train_tensor_y.pt",
    )
)

train_concept_mask_2 = torch.load(
    os.path.join(
        root,
        experiment,
        f"cov_{args.cov[-1]}",
        iteration,
        expert_type,
        output,
        "train_mask_alpha.pt",
    )
)

test_tensor_image_2 = torch.load(
    os.path.join(
        root,
        experiment,
        f"cov_{args.cov[-1]}",
        iteration,
        expert_type,
        output,
        "val_tensor_images.pt",
    )
)

test_tensor_y_2 = torch.load(
    os.path.join(
        root,
        experiment,
        f"cov_{args.cov[-1]}",
        iteration,
        expert_type,
        output,
        "val_tensor_y.pt",
    )
)

test_concept_mask_2 = torch.load(
    os.path.join(
        root,
        experiment,
        f"cov_{args.cov[-1]}",
        iteration,
        expert_type,
        output,
        "val_mask_alpha.pt",
    )
)


print("\n\n << Train sizes >>")
print(train_tensor_image_2.size())
print(train_tensor_y_2.size())
print(train_concept_mask_2.size())


print("\n\n << Test sizes >>")
print(test_tensor_image_2.size())
print(test_tensor_y_2.size())
print(test_concept_mask_2.size())



########################
0.01
0.2


 << Train sizes >>
torch.Size([1009, 3, 299, 299])
torch.Size([1009])
torch.Size([1009, 8])


 << Test sizes >>
torch.Size([231, 3, 299, 299])
torch.Size([231])
torch.Size([231, 8])


<IPython.core.display.Javascript object>

In [9]:
base_lr = 0.01
base_cov = 0.2
pickle_in = open(
    os.path.join(
        f"/ocean/projects/asc170022p/shg121/PhD/ICLR-2022/out/HAM10k/explainer/lr_{base_lr}_epochs_500_temperature-lens_0.7_input-size-pi_2048_cov_{base_cov}_alpha_0.5_selection-threshold_0.5_lambda-lens_0.0001_alpha-KD_0.9_temperature-KD_10.0_hidden-layers_1",
        "cov_0.2",
        "iter3",
        "explainer/accuracy",
        "test_explainer_configs.pkl",
    ),
    "rb",
)
args = pickle.load(pickle_in)
n_classes = len(args.labels)
x_to_bool = 0.5
top_k_explanations = 50
concept_names = args.concept_names
print("########################")
print(args.lr[-1])
print(args.cov[-1])

use_concepts_as_pi_input = True
explainer_init = "none"

experiment_folder = (
    f"lr_{args.lr[0]}_epochs_{args.epochs}_temperature-lens_{args.temperature_lens}"
    f"_input-size-pi_{args.input_size_pi}"
    f"_cov_{args.cov[0]}_alpha_{args.alpha}_selection-threshold_{args.selection_threshold}"
    f"_lambda-lens_{args.lambda_lens}_alpha-KD_{args.alpha_KD}"
    f"_temperature-KD_{float(args.temperature_KD)}_hidden-layers_{len(args.hidden_nodes)}"
)

root = "/ocean/projects/asc170022p/shg121/PhD/ICLR-2022/out/HAM10k"
experiment = f"explainer/{experiment_folder}"
iteration = "iter3"
expert_type = "explainer/accuracy"
output = "g_outputs"

# val
train_tensor_image_3 = torch.load(
    os.path.join(
        root,
        experiment,
        f"cov_{args.cov[-1]}",
        iteration,
        expert_type,
        output,
        "train_tensor_images.pt",
    )
)

train_tensor_y_3 = torch.load(
    os.path.join(
        root,
        experiment,
        f"cov_{args.cov[-1]}",
        iteration,
        expert_type,
        output,
        "train_tensor_y.pt",
    )
)

train_concept_mask_3 = torch.load(
    os.path.join(
        root,
        experiment,
        f"cov_{args.cov[-1]}",
        iteration,
        expert_type,
        output,
        "train_mask_alpha.pt",
    )
)

test_tensor_image_3 = torch.load(
    os.path.join(
        root,
        experiment,
        f"cov_{args.cov[-1]}",
        iteration,
        expert_type,
        output,
        "val_tensor_images.pt",
    )
)

test_tensor_y_3 = torch.load(
    os.path.join(
        root,
        experiment,
        f"cov_{args.cov[-1]}",
        iteration,
        expert_type,
        output,
        "val_tensor_y.pt",
    )
)

test_concept_mask_3 = torch.load(
    os.path.join(
        root,
        experiment,
        f"cov_{args.cov[-1]}",
        iteration,
        expert_type,
        output,
        "val_mask_alpha.pt",
    )
)


print("\n\n << Train sizes >>")
print(train_tensor_image_3.size())
print(train_tensor_y_3.size())
print(train_concept_mask_3.size())


print("\n\n << Test sizes >>")
print(test_tensor_image_3.size())
print(test_tensor_y_3.size())
print(test_concept_mask_3.size())



########################
0.01
0.2


 << Train sizes >>
torch.Size([1156, 3, 299, 299])
torch.Size([1156])
torch.Size([1156, 8])


 << Test sizes >>
torch.Size([303, 3, 299, 299])
torch.Size([303])
torch.Size([303, 8])


<IPython.core.display.Javascript object>

In [10]:
base_lr = 0.01
base_cov = 0.2
pickle_in = open(
    os.path.join(
        f"/ocean/projects/asc170022p/shg121/PhD/ICLR-2022/out/HAM10k/explainer/lr_{base_lr}_epochs_500_temperature-lens_0.7_input-size-pi_2048_cov_{base_cov}_alpha_0.5_selection-threshold_0.5_lambda-lens_0.0001_alpha-KD_0.9_temperature-KD_10.0_hidden-layers_1",
        "cov_0.2",
        "iter4",
        "explainer/accuracy",
        "test_explainer_configs.pkl",
    ),
    "rb",
)
args = pickle.load(pickle_in)
n_classes = len(args.labels)
x_to_bool = 0.5
top_k_explanations = 50
concept_names = args.concept_names
print("########################")
print(args.lr[-1])
print(args.cov[-1])

use_concepts_as_pi_input = True
explainer_init = "none"

experiment_folder = (
    f"lr_{args.lr[0]}_epochs_{args.epochs}_temperature-lens_{args.temperature_lens}"
    f"_input-size-pi_{args.input_size_pi}"
    f"_cov_{args.cov[0]}_alpha_{args.alpha}_selection-threshold_{args.selection_threshold}"
    f"_lambda-lens_{args.lambda_lens}_alpha-KD_{args.alpha_KD}"
    f"_temperature-KD_{float(args.temperature_KD)}_hidden-layers_{len(args.hidden_nodes)}"
)

root = "/ocean/projects/asc170022p/shg121/PhD/ICLR-2022/out/HAM10k"
experiment = f"explainer/{experiment_folder}"
iteration = "iter4"
expert_type = "explainer/accuracy"
output = "g_outputs"

# val
train_tensor_image_4 = torch.load(
    os.path.join(
        root,
        experiment,
        f"cov_{args.cov[-1]}",
        iteration,
        expert_type,
        output,
        "train_tensor_images.pt",
    )
)

train_tensor_y_4 = torch.load(
    os.path.join(
        root,
        experiment,
        f"cov_{args.cov[-1]}",
        iteration,
        expert_type,
        output,
        "train_tensor_y.pt",
    )
)

train_concept_mask_4 = torch.load(
    os.path.join(
        root,
        experiment,
        f"cov_{args.cov[-1]}",
        iteration,
        expert_type,
        output,
        "train_mask_alpha.pt",
    )
)

test_tensor_image_4 = torch.load(
    os.path.join(
        root,
        experiment,
        f"cov_{args.cov[-1]}",
        iteration,
        expert_type,
        output,
        "val_tensor_images.pt",
    )
)

test_tensor_y_4 = torch.load(
    os.path.join(
        root,
        experiment,
        f"cov_{args.cov[-1]}",
        iteration,
        expert_type,
        output,
        "val_tensor_y.pt",
    )
)

test_concept_mask_4 = torch.load(
    os.path.join(
        root,
        experiment,
        f"cov_{args.cov[-1]}",
        iteration,
        expert_type,
        output,
        "val_mask_alpha.pt",
    )
)


print("\n\n << Train sizes >>")
print(train_tensor_image_4.size())
print(train_tensor_y_4.size())
print(train_concept_mask_4.size())


print("\n\n << Test sizes >>")
print(test_tensor_image_4.size())
print(test_tensor_y_4.size())
print(test_concept_mask_4.size())



########################
0.01
0.2


 << Train sizes >>
torch.Size([835, 3, 299, 299])
torch.Size([835])
torch.Size([835, 8])


 << Test sizes >>
torch.Size([203, 3, 299, 299])
torch.Size([203])
torch.Size([203, 8])


<IPython.core.display.Javascript object>

In [11]:
base_lr = 0.01
base_cov = 0.2
pickle_in = open(
    os.path.join(
        f"/ocean/projects/asc170022p/shg121/PhD/ICLR-2022/out/HAM10k/explainer/lr_{base_lr}_epochs_500_temperature-lens_0.7_input-size-pi_2048_cov_{base_cov}_alpha_0.5_selection-threshold_0.5_lambda-lens_0.0001_alpha-KD_0.9_temperature-KD_10.0_hidden-layers_1",
        "cov_0.2",
        "iter5",
        "explainer/accuracy",
        "test_explainer_configs.pkl",
    ),
    "rb",
)
args = pickle.load(pickle_in)
n_classes = len(args.labels)
x_to_bool = 0.5
top_k_explanations = 50
concept_names = args.concept_names
print("########################")
print(args.lr[-1])
print(args.cov[-1])

use_concepts_as_pi_input = True
explainer_init = "none"

experiment_folder = (
    f"lr_{args.lr[0]}_epochs_{args.epochs}_temperature-lens_{args.temperature_lens}"
    f"_input-size-pi_{args.input_size_pi}"
    f"_cov_{args.cov[0]}_alpha_{args.alpha}_selection-threshold_{args.selection_threshold}"
    f"_lambda-lens_{args.lambda_lens}_alpha-KD_{args.alpha_KD}"
    f"_temperature-KD_{float(args.temperature_KD)}_hidden-layers_{len(args.hidden_nodes)}"
)

root = "/ocean/projects/asc170022p/shg121/PhD/ICLR-2022/out/HAM10k"
experiment = f"explainer/{experiment_folder}"
iteration = "iter5"
expert_type = "explainer/accuracy"
output = "g_outputs"

# val
train_tensor_image_5 = torch.load(
    os.path.join(
        root,
        experiment,
        f"cov_{args.cov[-1]}",
        iteration,
        expert_type,
        output,
        "train_tensor_images.pt",
    )
)

train_tensor_y_5 = torch.load(
    os.path.join(
        root,
        experiment,
        f"cov_{args.cov[-1]}",
        iteration,
        expert_type,
        output,
        "train_tensor_y.pt",
    )
)

train_concept_mask_5 = torch.load(
    os.path.join(
        root,
        experiment,
        f"cov_{args.cov[-1]}",
        iteration,
        expert_type,
        output,
        "train_mask_alpha.pt",
    )
)

test_tensor_image_5 = torch.load(
    os.path.join(
        root,
        experiment,
        f"cov_{args.cov[-1]}",
        iteration,
        expert_type,
        output,
        "val_tensor_images.pt",
    )
)

test_tensor_y_5 = torch.load(
    os.path.join(
        root,
        experiment,
        f"cov_{args.cov[-1]}",
        iteration,
        expert_type,
        output,
        "val_tensor_y.pt",
    )
)

test_concept_mask_5 = torch.load(
    os.path.join(
        root,
        experiment,
        f"cov_{args.cov[-1]}",
        iteration,
        expert_type,
        output,
        "val_mask_alpha.pt",
    )
)


print("\n\n << Train sizes >>")
print(train_tensor_image_5.size())
print(train_tensor_y_5.size())
print(train_concept_mask_5.size())


print("\n\n << Test sizes >>")
print(test_tensor_image_5.size())
print(test_tensor_y_5.size())
print(test_concept_mask_5.size())



########################
0.01
0.2


 << Train sizes >>
torch.Size([562, 3, 299, 299])
torch.Size([562])
torch.Size([562, 8])


 << Test sizes >>
torch.Size([126, 3, 299, 299])
torch.Size([126])
torch.Size([126, 8])


<IPython.core.display.Javascript object>

In [12]:
base_lr = 0.01
base_cov = 0.2
pickle_in = open(
    os.path.join(
        f"/ocean/projects/asc170022p/shg121/PhD/ICLR-2022/out/HAM10k/explainer/lr_{base_lr}_epochs_500_temperature-lens_0.7_input-size-pi_2048_cov_{base_cov}_alpha_0.5_selection-threshold_0.5_lambda-lens_0.0001_alpha-KD_0.9_temperature-KD_10.0_hidden-layers_1",
        "cov_0.2",
        "iter6",
        "explainer/accuracy",
        "test_explainer_configs.pkl",
    ),
    "rb",
)
args = pickle.load(pickle_in)
n_classes = len(args.labels)
x_to_bool = 0.5
top_k_explanations = 50
concept_names = args.concept_names
print("########################")
print(args.lr[-1])
print(args.cov[-1])

use_concepts_as_pi_input = True
explainer_init = "none"

experiment_folder = (
    f"lr_{args.lr[0]}_epochs_{args.epochs}_temperature-lens_{args.temperature_lens}"
    f"_input-size-pi_{args.input_size_pi}"
    f"_cov_{args.cov[0]}_alpha_{args.alpha}_selection-threshold_{args.selection_threshold}"
    f"_lambda-lens_{args.lambda_lens}_alpha-KD_{args.alpha_KD}"
    f"_temperature-KD_{float(args.temperature_KD)}_hidden-layers_{len(args.hidden_nodes)}"
)

root = "/ocean/projects/asc170022p/shg121/PhD/ICLR-2022/out/HAM10k"
experiment = f"explainer/{experiment_folder}"
iteration = "iter6"
expert_type = "explainer/accuracy"
output = "g_outputs"

# val
train_tensor_image_6 = torch.load(
    os.path.join(
        root,
        experiment,
        f"cov_{args.cov[-1]}",
        iteration,
        expert_type,
        output,
        "train_tensor_images.pt",
    )
)

train_tensor_y_6 = torch.load(
    os.path.join(
        root,
        experiment,
        f"cov_{args.cov[-1]}",
        iteration,
        expert_type,
        output,
        "train_tensor_y.pt",
    )
)

train_concept_mask_6 = torch.load(
    os.path.join(
        root,
        experiment,
        f"cov_{args.cov[-1]}",
        iteration,
        expert_type,
        output,
        "train_mask_alpha.pt",
    )
)

test_tensor_image_6 = torch.load(
    os.path.join(
        root,
        experiment,
        f"cov_{args.cov[-1]}",
        iteration,
        expert_type,
        output,
        "val_tensor_images.pt",
    )
)

test_tensor_y_6 = torch.load(
    os.path.join(
        root,
        experiment,
        f"cov_{args.cov[-1]}",
        iteration,
        expert_type,
        output,
        "val_tensor_y.pt",
    )
)

test_concept_mask_6 = torch.load(
    os.path.join(
        root,
        experiment,
        f"cov_{args.cov[-1]}",
        iteration,
        expert_type,
        output,
        "val_mask_alpha.pt",
    )
)


print("\n\n << Train sizes >>")
print(train_tensor_image_6.size())
print(train_tensor_y_6.size())
print(train_concept_mask_5.size())


print("\n\n << Test sizes >>")
print(test_tensor_image_6.size())
print(test_tensor_y_6.size())
print(test_concept_mask_6.size())



########################
0.01
0.2


 << Train sizes >>
torch.Size([552, 3, 299, 299])
torch.Size([552])
torch.Size([562, 8])


 << Test sizes >>
torch.Size([154, 3, 299, 299])
torch.Size([154])
torch.Size([154, 8])


<IPython.core.display.Javascript object>

In [16]:
train_tensor_images = torch.cat(
    (train_tensor_image_1, train_tensor_image_2, train_tensor_image_3, train_tensor_image_4, 
     train_tensor_image_5, train_tensor_image_6), dim=0
)
train_tensor_y = torch.cat(
    (train_tensor_y_1, train_tensor_y_2, train_tensor_y_3, train_tensor_y_4,
    train_tensor_y_5, train_tensor_y_6), dim=0
)
train_mask_alpha = torch.cat(
    (train_concept_mask_1, train_concept_mask_2, train_concept_mask_3, train_concept_mask_4, train_concept_mask_5,
    train_concept_mask_6), dim=0
)

print(train_tensor_images.size())
print(train_tensor_y.size())
print(train_mask_alpha.size())


torch.Size([7222, 3, 299, 299])
torch.Size([7222])
torch.Size([7222, 8])


<IPython.core.display.Javascript object>

In [19]:
test_tensor_images = torch.cat(
    (test_tensor_image_1, test_tensor_image_2, test_tensor_image_3, test_tensor_image_4,
    test_tensor_image_5, test_tensor_image_6), dim=0
)
test_tensor_y = torch.cat(
    (test_tensor_y_1, test_tensor_y_2, test_tensor_y_3, test_tensor_y_4, test_tensor_y_5, test_tensor_y_6), dim=0
)
test_mask_alpha = torch.cat(
    (test_concept_mask_1, test_concept_mask_2, test_concept_mask_3, test_concept_mask_4, 
     test_concept_mask_5, test_concept_mask_6), dim=0
)

print(test_tensor_images.size())
print(test_tensor_y.size())
print(test_mask_alpha.size())


torch.Size([1816, 3, 299, 299])
torch.Size([1816])
torch.Size([1816, 8])


<IPython.core.display.Javascript object>

In [20]:
save_path = f"/ocean/projects/asc170022p/shg121/PhD/ICLR-2022/out/{args.dataset}/completeness/{args.arch}/dataset"
os.makedirs(save_path, exist_ok=True)
print(save_path)
torch.save(train_tensor_images, os.path.join(save_path, f"train_tensor_images.pt"))
torch.save(train_tensor_y, os.path.join(save_path, f"train_tensor_y.pt"))
torch.save(train_mask_alpha, os.path.join(save_path, f"train_mask_alpha.pt"))

torch.save(test_tensor_images, os.path.join(save_path, f"test_tensor_images.pt"))
torch.save(test_tensor_y, os.path.join(save_path, f"test_tensor_y.pt"))
torch.save(test_mask_alpha, os.path.join(save_path, f"test_mask_alpha.pt"))

/ocean/projects/asc170022p/shg121/PhD/ICLR-2022/out/HAM10k/completeness/Inception_V3/dataset


<IPython.core.display.Javascript object>